In [1]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import AdaBoostClassifier

import pandas as pd
import numpy as np
import json

In [2]:
alpha_df = pd.read_csv('../Datos/Generados/04train.csv', index_col=False)
beta_df = pd.read_csv('../Datos/Generados/04test.csv', index_col=False)

In [3]:
alpha_df.head()

,description,cat_sub,category,subcategory,desc
0,sandalia havaianas top disney amar banana 35/36,21_22,21,22,sandaliahavaianas havaianastop sandaliatop san...
1,sandalia havaianas top max cinza aco 45/46,21_22,21,22,sandaliahavaianas havaianastop sandaliatop san...
2,sand havaianas sl org 33/34 areia,21_22,21,22,sandhavaianas havaianassl sandsl sandareia san...
3,sand (f) havaianas t max st 41/42 bc/mar,21_22,21,22,sandfemale femalehavaianas sandhavaianas sandm...
4,sandalia havaianas surf bg/pt 39x40,21_22,21,22,sandaliahavaianas havaianassurf sandaliasurf s...


In [4]:
# Assign dataframes division
train_df = alpha_df
test_df = beta_df

In [5]:
# Count and TFIDF
cv = CountVectorizer()
cv_fit = cv.fit_transform(train_df['desc'])

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(cv_fit)

In [6]:
X_train_tfidf.shape

(10358, 39890)

In [7]:
cv_fit.shape

(10358, 39890)

In [8]:
# SVM
clf = LinearSVC(loss='squared_hinge', C=10, random_state=42, max_iter=10000)
model = clf.fit(X_train_tfidf, train_df['cat_sub'])

In [9]:
# Apply model on test dataset
docs_new = test_df['desc']
X_new_counts = cv.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [10]:
# Predict
predicted = model.predict(X_new_tfidf)

In [11]:
test_df['t1'] = predicted
test_df[['cat','sub']] = test_df['t1'].str.split("_",expand=True)
test_df[['category','subcategory']] = test_df['cat_sub'].str.split("_",expand=True)

In [12]:
print(np.mean(predicted == test_df['cat_sub']), '- CATEGORIA Y SUBCATEGORIA')
print(np.mean(test_df['category'] == test_df['cat']), '- CATEGORIA')
print(np.mean(test_df['subcategory'] == test_df['sub']), '- SUBCATEGORIA')

0.7193373959316797 - CATEGORIA Y SUBCATEGORIA
0.8892798901381855 - CATEGORIA
0.7454295768603554 - SUBCATEGORIA
